# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')

bs4 or scrapy?

In [2]:
from typing import List, Tuple
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin
from azure.ai.projects.models import MessageRole, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import sys
import logging
sys.path.append(os.path.abspath('..'))  # Adjust the path as necessary
from utils.search_utils import web_search

# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
BING_GROUNDING_PROJECT_CONNECTION_STRING = os.getenv("BING_GROUNDING_PROJECT_CONNECTION_STRING")
BING_GROUNDING_AGENT_ID = os.getenv("BING_GROUNDING_AGENT_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
# Web search mode: "google" or "bing"
# it can be changed when users want to use different search engine
WEB_CRAWLING_MODE = os.getenv("WEB_CRAWLING_MODE") #on or off

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text



async def extract_contexts_async(url_snippet_tuples: List[Tuple[str, str]]) -> List[str]:
    """
    Asynchronously extract content from a list of URLs with their snippets.
    
    Args:
        url_snippet_tuples: List of (url, snippet) pairs to process
        
    Returns:
        List of extracted contents
    """
    async def fetch(url: str, snippet: str) -> str:
        # Try to get from cache first
        
        # If not in cache or cache unavailable, fetch the content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        
        try:
            async with httpx.AsyncClient(timeout=10, follow_redirects=True) as client:
                try:
                    response = await client.get(url, headers=headers)
                    response.raise_for_status()
                except httpx.HTTPStatusError as e:
                    # Handle redirects manually if needed
                    if e.response.status_code == 302 and "location" in e.response.headers:
                        redirect_url = e.response.headers["location"]
                        if not redirect_url.startswith("http"):
                            redirect_url = urljoin(url, redirect_url)
                        try:
                            response = await client.get(redirect_url, headers=headers)
                            response.raise_for_status()
                        except Exception as e2:
                            print(f"Redirect request failed: {e2}")
                            return f"{snippet} "
                    else:
                        print(f"Request failed: {e}")
                        return f"{snippet} "
                except httpx.HTTPError as e:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
                
                # Parse the content
                selector = scrapy.Selector(text=response.text)
                
                # Extract paragraphs
                paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
                
                # Remove duplicate and very short paragraphs
                filtered_paragraphs = []
                seen_content = set()
                for p in paragraphs:
                    # Skip very short paragraphs that are likely UI elements
                    if len(p) < 5:
                        continue
                    # Avoid duplicate content
                    if p in seen_content:
                        continue
                    seen_content.add(p)
                    filtered_paragraphs.append(p)
                
                # Join the filtered paragraphs
                text = "\n".join(filtered_paragraphs)
                
                # If no paragraphs were found, try to get other text content
                if not text:
                    content_texts = [t.strip() for t in selector.css(
                        'article::text, article *::text, .content::text, .content *::text, '
                        'main::text, main *::text'
                    ).getall() if t.strip()]
                    
                    if content_texts:
                        text = "\n".join(content_texts)
                
                # Combine snippet with extracted text
                snippet_text = f"{snippet}: {text}"
                
                
                return snippet_text
                
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            return f"{snippet} [Error: {str(e)}]"
    
    # Create tasks for all URLs
    tasks = [asyncio.create_task(fetch(url, snippet)) 
            for url, snippet in url_snippet_tuples]
    
    # Execute all tasks concurrently
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Process results
    processed_results = []
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Error processing URL {url_snippet_tuples[i][0]}: {str(result)}")
            processed_results.append(f"{url_snippet_tuples[i][1]} [Processing Error]")
        else:
            processed_results.append({"content": result, "url_citation" :{"link": url_snippet_tuples[i][0], "title": url_snippet_tuples[i][1]}})

    return processed_results
                    
           

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Web Search용 Query Rewrite:
            - 사용자의 질문을 실제 검색 엔진 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 웹 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"web_search": "웹 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())


In [3]:
from IPython.display import Markdown, display
from datetime import datetime
import time

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

async def process_web_search_call(RESULTS_COUNT, input, web_search_mode):
    
    start_time = time.time()
    
    contexts = [] 
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Web Search Query: {query_rewrite['web_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")
    
    results = web_search(query_rewrite, RESULTS_COUNT, web_search_mode=web_search_mode)
    print(f"Web Search Results: {len(results)}, {results}")
    if web_search_mode == "bing" and results and isinstance(results, list) and len(results) > 0:
        contexts = [results[i] for i in range(len(results))]
        
    elif web_search_mode == "google" and results and isinstance(results, list) and len(results) > 0:
        url_snippet_tuples = [(r["link"], r["snippet"]) for r in results]
        contexts = await extract_contexts_async(url_snippet_tuples)
        
    else:
        print("No results found or invalid response from web_search.")
        contexts = [] 
    
    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    
    
    system_prompt = """
        너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 
        답변은 마크다운으로 이모지를 1~2개 포함해서 작성해줘. 
        contexts를 최대한 활용하여 풍부하게 답변을 해야해. 
        사용자가 질문한 내용에 대해 정확하고 유용한 정보를 제공해야 해. contexts가 부족하면 최소한의 안내만 해줘. 
        url_citation은 사용자가 클릭할 수 있도록 링크를 제공해줘.
        
    """
    user_prompt = f"""
        너는 아래 제공하는 웹검색에서 검색한 contexts를 바탕으로 질문에 대한 답변을 제공해야 해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.
        웹검색에서 제공한 contexts: {contexts}
        질문: {query_rewrite['llm_query']}
        """
    
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds")


In [4]:
RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


web_search_mode = "google"

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input, web_search_mode)

web_search_mode = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input, web_search_mode)    

Google Search API 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 제품 중 2구 모델이 아닌 다른 인덕션 모델을 추천해 주세요. 특히 3구 이상 모델이나 다양한 화구 수를 가진 제품의 주요 특징과 장점을 설명해 주시면 감사하겠습니다.
Using web search mode: google
Web Search Results: 5, [{'kind': 'customsearch#result', 'title': '삼성 인덕션 설치 후기, 비스포크 3구 인덕션 추천 : 네이버 블로그', 'htmlTitle': '<b>삼성 인덕션</b> 설치 후기, 비스포크 <b>3구 인덕션 추천</b> : 네이버 블로그', 'link': 'https://blog.naver.com/y0umii/223295752049?viewType=pc', 'displayLink': 'blog.naver.com', 'snippet': 'Dec 18, 2023 ... 걱정되는건 오븐렌지와 상판 사이에 인덕션 설치 깊이가 확보되느냐? 였는데요. 최소 5cm 이상은 되어야 빌트인 설치가 가능하다고 합니다. \u200b.', 'htmlSnippet': 'Dec 18, 2023 <b>...</b> 걱정되는건 오븐렌지와 상판 사이에 <b>인덕션</b> 설치 깊이가 확보되느냐? 였는데요. 최소 5cm <b>이상</b>은 되어야 빌트인 설치가 가능하다고 합니다. \u200b.', 'formattedUrl': 'https://blog.naver.com/y0umii/223295752049?viewType=pc', 'htmlFormattedUrl': 'https://blog.naver.com/y0umii/223295752049?viewType=pc', 'pagemap': {'cse_thumbnail': [{'src': 'htt

삼성전자의 인덕션 제품 중 3구 이상의 모델로 추천할 만한 제품은 **삼성 Bespoke 인덕션**입니다. 이 제품은 다양한 화구 수와 여러 가지 뛰어난 특징을 가지고 있어 많은 소비자들에게 인기를 끌고 있습니다. 😊

### 주요 특징 및 장점

1. **다양한 화구 수**: 
   - 이 모델은 최소 3구 이상으로 제공되어 여러 가지 요리를 동시에 할 수 있는 편리함을 제공합니다. 

2. **고출력 화구**:
   - 가스레인지 이상의 화력을 원하신다면, 큰 화구의 출력이 **2,700W 이상**이어야 합니다. 삼성의 인덕션 모델은 이러한 높은 출력으로 빠른 조리가 가능하답니다. 🔥

3. **스마트 기술 연동**:
   - **스마트 쿡** 기능이 탑재되어 있어, **SmartThings 앱**과 연동하여 요리 과정을 보다 쉽게 관리할 수 있습니다. 앱을 통해 다양한 레시피를 참고하고 자동 조리가 가능합니다.

4. **원스톱 설치 서비스**:
   - 삼성전자는 설치가 복잡하지 않도록 **원스톱 설치 서비스**를 제공하여, 가스막음, 전기공사 및 싱크대 설치까지 전문 설치팀이 진행합니다. 

5. **안전 기능**:
   - 진동감지센서가 장착되어 있어 물이 끓는 상황을 감지하고 자동으로 화력을 조절하여 안전한 요리를 도와줍니다.

6. **세련된 디자인**:
   - 고강도 상판과 세련된 디자인으로 주방 인테리어와 잘 어울립니다.

더욱 자세한 정보를 원하신다면, [삼성 인덕션 공식 웹사이트](https://www.samsung.com/sec/cooking-appliances/)를 방문해 보세요! 

이와 같은 특징들을 가진 삼성 인덕션 제품은 가족의 다양한 요리 스타일을 지원하며, 효율적인 주방 환경을 만들어 줍니다.

elapsed time: 9.82 seconds
Bing Grounding 검색 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 설명해 주시면 좋겠습니다.
Using web search mode: bing
Web Search Results: 6, [{'content': 'Here are the top 5 relevant results for Samsung induction cooktops with 3 or more burners, along with their features and advantages:\n\n1. **NZ63DG403CFK**: Features simultaneous use of 3 burners with automatic power adjustment within 3,400W. Includes MAX Boost for high-power cooking【3:0†source】.\n2. **BESPOKE AI Induction Infinite Line (NZ63DB857CAV)**: Offers flexible zones for versatile cooking, automatic power distribution, and MAX Boost for enhanced performance【3:1†source】.\n3. **NZ63B5056AK**: Equipped with Quattro Flex Zone for large cookware, delivering up to 3,400W power output【3:4†source】.\n4. **BESPOKE AI Induction (NZ63DB607CABS)**: Includes Smart 

삼성전자의 3구 이상 인덕션 제품에 대한 정보를 아래에 정리해 드릴게요! 🍳✨

1. **NZ63DG403CFK**
   - **주요 기능**: 3개의 화구를 동시에 사용할 수 있으며, 자동 전력 조정 기능이 있어 최대 3,400W의 출력을 지원합니다. 
   - **장점**: MAX Boost 기능으로 고출력 요리가 가능해 빠르게 조리할 수 있습니다. 
   - [더 알아보기](https://www.samsung.com/sec/ai-subs-kitchen/NZ63DG403CFK-smart/NZ63DG403CFK/)

2. **BESPOKE AI Induction Infinite Line (NZ63DB857CAV)**
   - **주요 기능**: 플렉스 존을 통해 유연한 요리가 가능하며, 자동 전력 분배 기능과 MAX Boost를 지원합니다.
   - **장점**: 다양한 조리 용기에 맞춰 최적의 조리를 지원합니다.
   - [더 알아보기](https://www.samsung.com/sec/electric-range/cooktop-nz63db857cav-d2c/NZ63DB857CAV/)

3. **NZ63B5056AK**
   - **주요 기능**: Quattro Flex Zone이 장착되어 있어 대형 조리기구도 쉽게 사용할 수 있으며, 최대 3,400W의 출력을 제공합니다.
   - **장점**: 넓은 조리 공간 덕분에 여러 요리를 동시에 준비할 수 있습니다.
   - [더 알아보기](https://www.samsung.com/sec/electric-range/cooktop-nz63b5056cb-d2c/NZ63B5056AK/)

4. **BESPOKE AI Induction (NZ63DB607CABS)**
   - **주요 기능**: 스마트 제어 기능으로 원격 조작이 가능하며, 안전 기능도 포함되어 있습니다.
   - **장점**: 요리 중에도 언제든지 확인하고 조정할 수 있어 편리합니다.
   - [더 알아보기](https://www.samsung.com/sec/business/electric-range/cooktop-nz63db607cas-d2c/NZ63DB607CABS/)

5. **BESPOKE Infinite Line (NZ64B9899RA)**
   - **주요 기능**: All-Flex Zone이 있어 제한 없는 조리 공간을 제공하며, 모든 화구를 동시에 사용할 수 있고 최대 7,400W 출력을 지원합니다.
   - **장점**: 여러 요리를 동시에 만들 수 있어 대가족이나 손님 초대 시 매우 유용합니다.
   - [더 알아보기](https://www.samsung.com/sec/ai-subs-kitchen/NZ64B9899RA-smart/NZ64B9899RA/)

각 모델마다 독특한 기능과 장점이 있으니, 필요에 맞는 모델을 선택해 보세요! 😊

elapsed time: 20.89 seconds


In [5]:

RESULTS_COUNT = 5

inputs = [
    #"삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


web_search_mode = "google"

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input, web_search_mode=web_search_mode)

web_search_mode = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input, web_search_mode=web_search_mode)    

Google Search API 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Web Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물할 삼성전자 TV를 추천해 주세요. 다양한 모델 중에서 특히 추천할 만한 이유와 함께 주요 기능을 설명해 주시면 좋겠습니다.
Using web search mode: google
Web Search Results: 5, [{'kind': 'customsearch#result', 'title': '다가오는 설 맞이 부모님 효도 선물로 안성맞춤, 삼성전자 43형 풀HD ...', 'htmlTitle': '다가오는 설 맞이 <b>부모님</b> 효도 <b>선물</b>로 안성맞춤, <b>삼성전자</b> 43형 풀HD ...', 'link': 'https://blog.naver.com/danawaelec/220913253027?viewType=pc', 'displayLink': 'blog.naver.com', 'snippet': 'Jan 17, 2017 ... 곧 다가올 설 명절을 맞이하여 부모님 효도 선물로 좋은 TV는 무엇일까? 선물용으로 화질이 좋으면서 A/S 걱정까지 없어야 하기에 많은 이들이 대기업 HD\xa0...', 'htmlSnippet': 'Jan 17, 2017 <b>...</b> 곧 다가올 설 명절을 맞이하여 <b>부모님</b> 효도 <b>선물</b>로 좋은 <b>TV</b>는 무엇일까? 선물용으로 화질이 좋으면서 A/S 걱정까지 없어야 하기에 많은 이들이 대기업 HD&nbsp;...', 'formattedUrl': 'https://blog.naver.com/danawaelec/220913253027?viewType=pc', 'htmlFormattedUrl': 'https://blog.naver.com/danawaelec/220913253027?viewType=pc', 'pagemap': {

부모님께 선물할 삼성전자 TV로는 **UN49MU6250F**와 **UN50MU6350F**를 추천드립니다. 두 모델 모두 UHD 해상도를 지원하며, 화질과 가격 면에서 뛰어난 가성비를 자랑합니다. 😊📺

### 1. 삼성전자 UN49MU6250F
- **해상도**: 4K UHD
- **가격**: 약 68만원대 (할인 및 행사 시 더 저렴할 수 있습니다)
- **주요 기능**:
  - 고화질 영상: 4K 해상도로 선명한 화질 제공
  - 스마트 기능: 다양한 스트리밍 서비스와 앱을 지원하여 쉽게 콘텐츠를 즐길 수 있음
  - 연결성: HDMI 및 USB 포트를 통한 다양한 기기 연결 가능
- **추천 이유**: 가격이 합리적이며, 부모님께서 TV를 이용해 다양한 프로그램을 시청하시기에 적합한 모델입니다. 어버이날이나 명절 선물로 매우 좋은 선택이 될 것입니다. [자세히 보기](https://blog.naver.com/danawaelec/221259552788?viewType=pc)

### 2. 삼성전자 UN50MU6350F
- **해상도**: 4K UHD
- **주요 기능**:
  - HDR 지원: 더 넓은 색 영역과 대비로 더 생생한 화면 제공
  - 스마트 TV 기능: 인터넷에 연결하여 다양한 콘텐츠 접근 가능
  - 음성 인식: 리모컨을 통해 음성 명령으로 TV 조작 가능
- **추천 이유**: 중급형 라인업으로 가격 대비 우수한 성능을 가지고 있어 부모님께서 사용하기에도 무리가 없습니다. 더욱이 사용이 간편한 스마트 기능이 있어 세대간의 기술 격차를 줄일 수 있습니다. [자세히 보기](https://blog.naver.com/danawaelec/221198055323?viewType=pc)

이 외에도 삼성전자는 다양한 TV 모델을 보유하고 있으니, 부모님의 사용 패턴과 취향에 맞는 제품을 선택하는 것이 중요합니다!

elapsed time: 7.26 seconds
Google Search API 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 2025년 제품 작년 대비 개선된 점
LLM Query: 삼성전자가 2025년에 출시한 제품이 작년 모델에 비해 어떤 점에서 개선되었는지 구체적으로 설명해 주세요. 주요 기능, 성능 향상, 디자인 변화 등을 포함해서 알려주시면 좋겠습니다.
Using web search mode: google
Web Search Results: 5, [{'kind': 'customsearch#result', 'title': '2월 10일의 거시경제 Letter : 네이버 블로그', 'htmlTitle': '2월 10일의 거시경제 Letter : 네이버 블로그', 'link': 'https://blog.naver.com/james_lee_advisors/223755533576', 'displayLink': 'blog.naver.com', 'snippet': 'Feb 10, 2025 ... 특히 독일과 영국의 1월 전기차 판매가 전년 대비 큰 폭으로 증가했다는 점에서 지속 가능성이 주목됩니다. ... 특히 삼성전자는 2022년 이후 3년 만에\xa0...', 'htmlSnippet': 'Feb 10, 2025 <b>...</b> 특히 독일과 영국의 1월 전기차 판매가 <b>전년 대비</b> 큰 폭으로 증가했다는 <b>점</b>에서 지속 가능성이 주목됩니다. ... 특히 <b>삼성전자</b>는 2022<b>년</b> 이후 3<b>년</b> 만에&nbsp;...', 'formattedUrl': 'https://blog.naver.com/james_lee_advisors/223755533576', 'htmlFormattedUrl': 'https://blog.naver.com/james_lee_advisors/223755533576', 'page

삼성전자가 2025년에 출시한 제품들은 여러 가지 측면에서 작년 모델에 비해 개선되었습니다. 특히 **5세대 고대역폭 메모리(HBM)**인 **HBM3E**가 주목할 만한 혁신을 이끌고 있습니다. 📈

### 주요 기능 및 성능 향상
1. **고대역폭**: HBM3E는 이전 세대보다 대역폭이 크게 향상되었습니다. 이로 인해 데이터 전송 속도가 빨라져, 고사양 게임이나 AI 연산 처리에서 더욱 뛰어난 성능을 발휘합니다.
2. **에너지 효율성**: 새로운 메모리 기술이 적용되어 전력 소비를 줄이면서도 성능을 유지할 수 있는 점이 특징입니다. 이는 특히 데이터 센터나 고성능 컴퓨팅 환경에서 매우 중요합니다.

### 디자인 변화
- **모듈 디자인 개선**: HBM3E 모듈의 설계가 더욱 컴팩트해졌습니다. 이러한 변화는 설치 공간을 줄이고 시스템의 전체적인 효율성을 높이는 데 기여합니다.

### 경쟁력 강화
- 삼성전자는 SK하이닉스와의 경쟁에서도 우위를 점하고 있으며, 2025년부터 중국 시장에서의 사업 확장도 기대되고 있습니다. 이는 새로운 기술이 상용화되면서 더 큰 시장 점유율을 확보할 수 있는 기반이 됩니다. 🌏

이처럼 삼성전자는 2025년에 출시한 제품들을 통해 성능, 효율성 및 디자인 면에서 많은 개선을 이루어냈으며, 이러한 혁신은 고객들에게 더 나은 경험을 제공할 것입니다.

더욱 자세한 내용을 알고 싶으시면 [여기](https://blog.naver.com/irasu2/223744428165)를 클릭해 주세요!

elapsed time: 6.30 seconds
Bing Grounding 검색 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Web Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님에게 선물할 삼성전자 TV를 추천해 주세요. 특히 화질, 크기, 기능 등을 고려해서 가장 적합한 모델을 알려주시고, 각 모델의 장점도 설명해 주세요.
Using web search mode: bing
Web Search Results: 6, [{'content': "Here are the top 5 relevant results for your query:\n\n1. **Samsung & LG TV Comparison (2025)** - Discusses Samsung's QLED 4K UHD TVs with sizes ranging from 43 to 85 inches, highlighting features like smart apps and voice control【3:0†source】.\n2. **2025 Samsung AI TV Overview** - Covers the latest Neo QLED, OLED, and QLED models with advanced AI features and sound enhancements【3:1†source】.\n3. **Samsung TV 2025 Models Comparison** - Reviews 40 models, including Neo QLED 8K QN990F with AI upscaling and HDR remastering【3:2†source】.\n4. **Samsung TV Reviews & Comparison (2025)** - Recommends Neo QLED 4K TVs for their exceptional picture quality【3:3†source】.\n5. **Samsung & 

부모님께 선물할 삼성전자 TV를 추천해 드리겠습니다! 😊📺

### 1. **Samsung Neo QLED 8K QN990F**
- **화질**: 8K 해상도로, 놀라운 디테일을 제공하며 HDR 리마스터링 기능이 있어 더욱 선명한 색감을 경험할 수 있습니다.
- **크기**: 다양한 사이즈(43~85인치)로 제공되어, 집의 공간에 맞게 선택할 수 있습니다.
- **기능**: AI 업스케일링 기능이 있어 기존 콘텐츠도 더욱 고해상도로 감상할 수 있으며, 음성 제어가 가능한 스마트 기능이 있어 사용이 편리합니다.
- **장점**: 최상급 화질과 최신 AI 기술이 결합되어, 영화나 스포츠를 즐길 때 최적의 경험을 제공합니다.

### 2. **Samsung Neo QLED 4K**
- **화질**: 4K 해상도로 생생한 색감을 제공하며, 최신 화질 보정 기술을 적용하여 다양한 콘텐츠에서 최상의 시청 환경을 제공합니다.
- **크기**: 43인치부터 85인치까지 다양한 선택지가 있어 부모님이 선호하는 크기를 선택할 수 있습니다.
- **기능**: 스마트 TV 기능을 통해 넷플릭스, 유튜브 등의 다양한 앱을 쉽게 사용할 수 있습니다.
- **장점**: 높은 가성비와 함께 뛰어난 화질로, 일상적인 사용에 매우 적합합니다.

### 3. **Samsung QLED 4K**
- **화질**: QLED 패널을 사용하여 깊은 블랙과 밝은 색상을 동시에 구현합니다.
- **크기**: 다양한 크기로 제공되어, 거실이나 방에 맞게 선택할 수 있습니다.
- **기능**: 스마트 기능이 포함되어 있으며, 게임 모드와 같은 다양한 옵션으로 엔터테인먼트 경험을 향상시킵니다.
- **장점**: 가격 대비 뛰어난 화질과 기능으로, 가성비가 우수한 모델입니다.

각 모델에 대한 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다:
- [삼성 & LG 비교 TOP 5 (2025 순위)](https://choicemon.com/best-tv/)
- [2025 삼성 AI TV 전 모델 정리](https://ajossi-j.tistory.com/51)
- [삼성전자 2025년형 TV 40종 비교](https://monna2.tistory.com/531)
- [2025년 삼성전자 TV 리뷰 & 비교](https://nosearch.com/recommendation/pick/tv/tv)
- [삼성전자·LG전자 TV 비교](https://blog.naver.com/tearhunter/223798058907)

부모님께서 만족하실 TV 모델을 잘 선택하시길 바랍니다! 🎉

elapsed time: 16.93 seconds
Bing Grounding 검색 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 25년 제품 작년 대비 개선 사항
LLM Query: 삼성전자의 2025년 모델 제품이 작년 모델에 비해 어떤 점에서 개선되었는지 알려주세요. 각 개선 사항의 주요 특징과 장점을 포함해 주시면 좋겠습니다.
Using web search mode: bing
Web Search Results: 6, [{'content': "Here are the top 5 relevant results about improvements in Samsung's 2025 models compared to 2024:\n\n1. **Samsung's 2025 Galaxy A Series and Z Flip FE**: Enhanced performance, innovative features, and attractive pricing【3:0†source】.\n2. **Galaxy S25 Series Overview**: Slimmer design, integrated S Pen, advanced AI-enhanced zoom, and improved multitasking【3:1†source】.\n3. **2025 Galaxy Product Lineup**: Includes Galaxy S25 Ultra, Fold7, and other flagship models【3:2†source】.\n4. **Galaxy S25 and Z Flip 6 Innovations**: Superior camera, larger external screen, and new color options【3:4†source】.\n5. **Samsung's 2025 AI TV Lineup**: Features ultra-large screens and AI integrati

삼성전자의 2025년 모델은 2024년 모델에 비해 여러 가지 개선 사항이 도입되었습니다. 아래에 주요 특징과 장점을 정리해 드릴게요! 📱✨

1. **Galaxy A 시리즈와 Z Flip FE**:
   - **향상된 성능**: 더 빠르고 효율적인 프로세서로 사용자의 경험을 개선.
   - **혁신적인 기능**: 카메라 성능, 배터리 수명 등이 강화되어 일상에서의 사용이 편리해졌습니다.
   - **매력적인 가격**: 소비자 친화적인 가격으로 고급 기능을 제공합니다.
   - [자세히 보기](https://heoc-life.tistory.com/76)

2. **Galaxy S25 시리즈**:
   - **슬림한 디자인**: 보다 세련된 외관과 손에 쥐기 좋은 그립감을 제공.
   - **통합된 S Pen**: 생산성과 창의성을 높여주는 기능으로, 필기 및 스케치가 더욱 편리해졌습니다.
   - **고급 AI 강화 줌**: 사진 및 영상 촬영 시 더욱 정교한 줌 기능을 제공, 멋진 순간을 놓치지 않을 수 있습니다.
   - [자세히 보기](https://blog.naver.com/PostView.naver?blogId=naramharam&logNo=223810258895)

3. **Galaxy S25 Ultra 및 Fold7**:
   - **우수한 카메라**: 고화소 카메라로 다양한 촬영 환경에서 뛰어난 성능을 발휘.
   - **더 큰 외부 화면**: 편리한 멀티태스킹과 알림 확인이 용이합니다.
   - **새로운 색상 옵션**: 개인의 취향을 반영할 수 있는 다양한 색상이 추가되었습니다.
   - [자세히 보기](https://r1.community.samsung.com/t5/%EA%B0%A4%EB%9F%AC%EC%8B%9C-s/2025%EB%85%84-%EA%B0%A4%EB%9F%AD%EC%8B%9C-%EC%A0%9C%ED%92%88-%EB%9D%BC%EC%9D%B8%EC%97%85/td-p/30353231)

4. **AI TV 라인업**:
   - **초대형 화면**: 보다 몰입감 있는 시청 경험을 제공합니다.
   - **AI 통합**: 개인 맞춤형 추천 및 최적화된 화면 설정으로 사용 편의성이 증가했습니다.
   - [자세히 보기](https://news.samsung.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-2025%EB%85%84%ED%98%95-tv-%EC%8B%A0%EC%A0%9C%ED%92%88-%EB%9D%BC%EC%9D%B8%EC%97%85-%EA%B3%B5%EA%B0%9C-%EC%A7%84%EC%A0%95%ED%95%9C-ai-tv-%EC%8B%9C%EB%8C%80-2)

이 외에도 여러 가지 개선 사항이 있지만, 주요 특징들을 통해 2025년 모델이 더 향상된 성능과 기능을 제공하고 있다는 점을 알 수 있습니다. 사용자가 원하는 다양한 요구를 충족시킬 수 있도록 설계되었어요! 😊

elapsed time: 21.85 seconds
